<a href="https://colab.research.google.com/github/hugomedsa/melhores-melodias-scrap/blob/main/Ranking_de_melhores_melodias_de_um_site.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas_datareader import data
from google.colab.data_table import DataTable
import lxml
import json

##### CIFRA MELÓDICA FAVORITAS/VISUALIZAÇÕES

In [ ]:
url = 'https://ciframelodica.com.br/musicas/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
musicas = soup.find('div', class_='container nopadding')
nomes = list(musicas.find_all('h1', class_='listaH1'))
dados = list(musicas.find_all('span', class_='label'))

table = {'Views':[],'Favorite':[]}
for i in range(0, 153,3):
  table['Views'].append(dados[i].text.replace(',',''))
  table['Favorite'].append(dados[i+1].text.replace(',',''))
tabela = pd.DataFrame(table).astype(int)

cancoes = {'Music':[]}
for i, c in enumerate(nomes): 
  cancoes['Music'].append(nomes[i].text)
titulos = pd.DataFrame(cancoes) 

tabela['Titulos'] = titulos['Music']
tabela['Melhores'] = tabela['Favorite'] / tabela['Views']
tabela.dropna().sort_values('Melhores',ascending=False)

,Views,Favorite,Titulos,Melhores
2,22,1,Don't Dream It's Over,0.045455
4,70,2,Tu és o meu Deus,0.028571
6,148,3,Snap,0.020270
41,148,3,Um Poema De Amor,0.020270
42,200,4,Bang Bang,0.020000
7,254,5,Stand proud,0.019685
13,209,4,Acorn Plains Overworld,0.019139
50,553,10,A lua e eu,0.018083
16,344,6,Leave the Door Open,0.017442
26,276,4,Nosso Xote,0.014493


#COM SELENIUM

In [ ]:
!pip install selenium --quiet
!apt-get update --quiet
!apt install chromium-chromedriver --quiet

In [ ]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import  Service

In [ ]:
options = webdriver.ChromeOptions()

options.add_argument("start-maximized")
options.add_argument("enable-automation")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--disable-infobars")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu")
options.add_argument("--disable-gl-drawing-for-tests") #EU QUE COLOQUEI, DIZ QUE ELIMINA SAÍDA PIXEL GL DE IMAGENS, PARA CARREGAR MAIS RAPIDO
options.add_argument('--blink-settings=imagesEnabled=false') #EU QUE COLOQUEI, PRA PARAR DE MOSTRAR IMAGENS
options.set_capability("pageLoadStrategy", "normal")

wd_chrome = webdriver.Chrome(options = options)

In [ ]:
wd_chrome.get('https://ciframelodica.com.br/musicas/') #REINICIAR O CHROME

In [ ]:
# RASPAGEM SIMPLES, COM XPATH MANUAL, COMO SE FOSSE REGEX - PODENDO VOLTAR DA ÚLTIMA PÁGINA
favs = wd_chrome.find_element("xpath",'//*[@id="controlRoom"]/div/div[2]/table/tbody/tr[1]/td[4]')
favs.click()
lista = []
#last_pg = wd_chrome.find_element_by_xpath('//*[@id="controlRoom"]/div/div[1]/div[2]/button[2]')
#last_pg.click()
for c in range (0,212): #PEGANDO 100 ÚLTIMAS PÁGINAS
  dados_brutos = wd_chrome.find_elements("xpath","//div[@class='container nopadding']/a")
  for c in dados_brutos:
    lista.append((f"{c.text}\n{c.get_attribute('href')}").split("\n"))
  #previous_pg = wd_chrome.find_element_by_xpath('//*[@id="controlRoom"]/div/div[1]/div[1]/button[2]')
  #previous_pg.click()
  next_pg = wd_chrome.find_element("xpath",'//*[@id="controlRoom"]/div/div[1]/div[2]/button[1]')
  next_pg.click()


In [ ]:
tabela = pd.DataFrame(lista,columns={'Música','Artista','Views','Gênero','Link'})
tabela

,Views,Artista,Música,Gênero,Link
0,Aleluia Hallelujah,Gabriela Rocha,"777,917 2,018 413",Evangélicas,https://ciframelodica.com.br/musicas/gabriela-...
1,Asa Branca,Luiz Gonzaga,"532,402 1,212 315",MPB,https://ciframelodica.com.br/musicas/luiz-gonz...
2,Titanic My Heart Will Go On,Celine Dion,"697,649 1,127 420",Filmes-TV,https://ciframelodica.com.br/musicas/celine-di...
3,Hino Nacional Brasileiro completo,Joaquim e Francisco,"563,270 1,099 338",Hinos,https://ciframelodica.com.br/musicas/joaquim-e...
4,A Pantera Cor de Rosa,Henry Mancini,"484,258 1,002 313",Filmes-TV,https://ciframelodica.com.br/musicas/henry-man...
...,...,...,...,...,...
11019,Be Right There,MattyB,"2,971 1 0",Pop Internacional,https://ciframelodica.com.br/musicas/mattyb-be...
11020,Nosso Deus,Salz Band,"6,374 1 0",Evangélicas,https://ciframelodica.com.br/musicas/salz-band...
11021,Bara wa Utsukushiku Chiru,Lady Oscar,"3,096 1 0",Anime,https://ciframelodica.com.br/musicas/lady-osca...
11022,HC 170 Ao Calvário de Horror,Harpa Cristã,"3,379 1 1",Evangélicas,https://ciframelodica.com.br/musicas/harpa-cri...


In [ ]:
Tabela = pd.DataFrame(lista,columns=['Musica','Cantor','Metricas','Estilo','Link'])
Tabela

In [ ]:
Tabela.to_excel('101_primeiras_pages_Cifra_Melodica.xlsx')